<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase3/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning do Modelo BERT com AmazonTitles-1.3MM

Neste notebook, realizaremos o fine-tuning do modelo BERT (`bert-base-uncased`) usando o dataset "The AmazonTitles-1.3MM". O objetivo é treinar o modelo para que ele consiga gerar descrições de produtos com base em seus títulos.

### 1. Instalar dependências

Primeira célula: Instala as bibliotecas necessárias.


In [1]:
# Instalar as bibliotecas necessárias
!pip install datasets transformers numpy scipy

## 2. Importar as Bibliotecas e criar as funções utilitárias

Agora, importaremos as bibliotecas necessárias para nosso trabalho. Além de criar funções para executar em nosso processo.

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch
import gc, os
import numpy as np
import scipy.sparse as sp
import pandas, gzip

def to_scipy_matrix(lol_inds, num_cols):
  cols = np.concatenate(lol_inds)
  rows = np.concatenate([[i]*len(x) for i, x in enumerate(lol_inds)])
  data = np.concatenate([[1.0]*len(x) for i, x in enumerate(lol_inds)])
  return sp.coo_matrix((data, (rows, cols)), (len(lol_inds), num_cols)).tocsr()

def process_lf_amazon_datasets(dataset):
  print('Reading raw dataset files...')
  trn_df = pandas.read_json(gzip.open(f'Datasets/{dataset}/trn.json.gz'), lines=True)
  tst_df = pandas.read_json(gzip.open(f'Datasets/{dataset}/tst.json.gz'), lines=True)
  lbl_df = pandas.read_json(gzip.open(f'Datasets/{dataset}/lbl.json.gz'), lines=True)

  print('Processing Y (label) files...')
  trn_X_Y = to_scipy_matrix(trn_df.target_ind.values, lbl_df.shape[0])
  tst_X_Y = to_scipy_matrix(tst_df.target_ind.values, lbl_df.shape[0])

  sp.save_npz(f'Datasets/{dataset}/Y.trn.npz', trn_X_Y)
  sp.save_npz(f'Datasets/{dataset}/Y.tst.npz', tst_X_Y)

  print('Processing X (input) files...')
  print(*trn_df.title.apply(lambda x: x.strip()).values, sep='\n', file=open(f'Datasets/{dataset}/raw/trn_X.txt', 'w'))
  print(*tst_df.title.apply(lambda x: x.strip()).values, sep='\n', file=open(f'Datasets/{dataset}/raw/tst_X.txt', 'w'))
  print(*lbl_df.title.apply(lambda x: x.strip()).values, sep='\n', file=open(f'Datasets/{dataset}/raw/Y.txt', 'w'))

  #print('Tokenizing X (input) files...')
  #max_len = 32 if 'titles' in dataset else 128
  #os.system(f"python utils/tokenization_utils.py --data-path Datasets/{dataset}/raw/trn_X.txt --tf-max-len {max_len}")
  #os.system(f"python utils/tokenization_utils.py --data-path Datasets/{dataset}/raw/tst_X.txt --tf-max-len {max_len}")
  #os.system(f"python utils/tokenization_utils.py --data-path Datasets/{dataset}/raw/Y.txt --tf-max-len {max_len}")

  return (trn_df, tst_df, lbl_df)

## 3. Baixar dados o Google Drive

Vamos baixar os dados do Google Drive para acessar os arquivos que contêm os dados de treinamento e teste.

In [3]:
!mkdir -p Datasets
!cd Datasets; gdown 12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK;
!cd Datasets; unzip LF-Amazon-1.3M.raw.zip; mkdir -p LF-AmazonTitles-1.3M/raw; mv LF-Amazon-1.3M/* LF-AmazonTitles-1.3M

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=e7e82f98-649f-4167-878e-9adca4b955bd
To: /home/guilherme/Datasets/LF-Amazon-1.3M.raw.zip
100%|████████████████████████████████████████| 890M/890M [01:01<00:00, 14.5MB/s]
Archive:  LF-Amazon-1.3M.raw.zip
   creating: LF-Amazon-1.3M/
  inflating: LF-Amazon-1.3M/lbl.json.gz  
  inflating: LF-Amazon-1.3M/trn.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_test.txt  
  inflating: LF-Amazon-1.3M/tst.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_train.txt  


## 4. Carregar o Dataset de Treinamento

Carregaremos o dataset de treinamento (`trn.json`) utilizando a biblioteca `datasets`.

In [ ]:
# Ler o arquivo jsonl com pandas
(dataset, tst_df , lbl_df) = process_lf_amazon_datasets('LF-AmazonTitles-1.3M')

# Exibir uma amostra dos dados
print(dataset.head())

Reading raw dataset files...


## 5. Processar o Dataset de Treinamento

Vamos processar os dados, criando prompts para o modelo com base nos títulos dos produtos e suas descrições.

In [ ]:
# Inicializar uma lista para armazenar os dados processados
dados_processados = []

# Iterar sobre o dataset e criar os pares de entrada e saída
for example in dataset:
    title = example.get('title', '')
    content = example.get('content', '')

    # Criar um prompt com base no título e na descrição
    prompt = f"Descreva o produto com o título '{title}'?"
    dados_processados.append({"input_text": prompt, "output_text": content})

    # Opcionalmente, pare após processar um certo número de exemplos
    # if len(dados_processados) >= 10000:
        # break  # Ajuste este valor conforme necessário para processar mais exemplos

# Mostrar uma amostra dos dados processados
dados_processados[:2]

## 6. Tokenizar os Dados de Treinamento em Lotes

Utilizaremos o `AutoTokenizer` da biblioteca `transformers` para tokenizar os dados de entrada e saída em lotes menores.


In [ ]:
# Carregar o tokenizer do BERT (bert-base-uncased) com uso rápido
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True, clean_up_tokenization_spaces=True)

batch_size = 16  # Tamanho do lote reduzido

def tokenize_in_batches(dataset, batch_size):
    tokenized_data = []
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]
        # Tokenização com limpeza de espaços controlada
        inputs = tokenizer(
            [example['input_text'] for example in batch],
            padding="max_length",
            truncation=True,
            max_length=256
        )
        tokenized_data.append(inputs)
    return tokenized_data

# Usar apenas um subconjunto para teste (opcional)
dados_processados = dados_processados[:100000]  # Use as primeiras 1000 amostras

# Tokenizar os dados processados
dados_tokenizados = tokenize_in_batches(dados_processados, batch_size)

# Limpar variáveis não necessárias
del dados_processados
gc.collect()

# Exemplo de um registro tokenizado
print(dados_tokenizados[0])

## 7. Criar o Dataset de Treinamento

Agora, criaremos o dataset de treinamento e dividiremos em conjuntos de treino e validação.

In [ ]:
from datasets import Dataset

# Converter os dados tokenizados em um Dataset da Hugging Face
hf_dataset = Dataset.from_list(dados_tokenizados)
train_test_split = hf_dataset.train_test_split(test_size=0.2)

## 8. Configurar e Treinar o Modelo

Iremos configurar o modelo BERT para o fine-tuning e definir os parâmetros de treinamento.

In [ ]:
# Carregar o modelo BERT para Masked Language Modeling
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    save_total_limit=2,
)

# Inicializar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
)

## 9. Treinar o Modelo

Agora vamos iniciar o treinamento do modelo.

In [ ]:
trainer.train()

## 10. Carregar o Conjunto de Teste

Em seguida, carregaremos o conjunto de teste (`tst.json`).

In [ ]:
tst_json_file = '/content/drive/My Drive/caminho_para_o_seu_arquivo/tst.json'  # Altere para o caminho correto
test_dataset = load_dataset('json', data_files=tst_json_file, split='train')

## 11. Processar o Conjunto de Teste

Processaremos o conjunto de teste da mesma forma que fizemos com o conjunto de treinamento.

In [ ]:
test_dados_processados = []
for example in test_dataset:
    title = example.get('title', '')
    prompt = f"Descreva o produto com o título '{title}'?"
    test_dados_processados.append({"input_text": prompt})

## 12. Tokenizar o Conjunto de Teste

Tokenizaremos o conjunto de teste para que possamos usá-lo na avaliação do modelo.

In [ ]:
test_dados_tokenizados = [tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=512) for example in test_dados_processados]

## 13. Avaliar o Modelo

Iremos avaliar o modelo utilizando o conjunto de teste que preparamos.

In [ ]:
def gerar_respostas(model, tokenized_inputs):
    model.eval()
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
    return outputs

respostas = []
for tokenized_input in test_dados_tokenizados:
    resposta = gerar_respostas(model, tokenized_input)
    respostas.append(resposta)

## 14. Exibir Algumas Respostas Geradas

Por fim, exibiremos algumas respostas geradas pelo modelo.

In [ ]:
print(respostas[:5])